In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from __future__ import division
from __future__ import print_function
import scipy
import os
import glob
import time
import random
import argparse
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import scipy.sparse as sp
from utils import *
from write_mandatory_files_for_adsf import *
from rwr_process import *

In [14]:
# write_shortest_path_adj_matrix('cora')
# write_shortest_path_adj_matrix('citeseer')
# write_shortest_path_adj_matrix('pubmed')



2708it [00:26, 101.14it/s]


File saved at data/adsf/dijskra_cora.pkl


3327it [00:20, 159.60it/s]


File saved at data/adsf/dijskra_citeseer.pkl


19717it [31:34, 10.41it/s]


File saved at data/adsf/dijskra_pubmed.pkl


In [7]:
dataset = "citeseer"
adj, features,  idx_train, idx_val, idx_test, train_mask, val_mask, test_mask,labels ,adj_ad= load_data_orggcn(dataset)
dataset_len = features.shape[0]

node number: 3327
training sample: 120
validation sample: 500
testing sample: 1000


100%|██████████| 1/1 [00:00<?, ?it/s]


In [11]:
model =  RWR_process(nfeat=features.shape[1],
                nhid=8,
                nclass=int(labels.max()) + 1,
                dropout=0.6,
                nheads=8,
                alpha=0.2,
                adj_ad=0, dataset=dataset, dataset_len=dataset_len)


In [12]:
x_dense = features.todense() if sp.issparse(features) else features

# Then, convert this dense matrix to a PyTorch Tensor
x_tensor = torch.FloatTensor(x_dense)

In [13]:
model.to('cuda')
model.forward(x_tensor.to('cuda'), adj.to('cuda'),0)

File saved at data/adsf/ri_index_c_0.5_citeseer_highorder_1_x_abs.pkl
File saved at data/adsf/ri_all_c_0.5_citeseer_highorder_1_x_abs.pkl
File saved at data/adsf/ri_index_c_0.5_citeseer_highorder_1_x_abs.pkl
File saved at data/adsf/ri_all_c_0.5_citeseer_highorder_1_x_abs.pkl
File saved at data/adsf/ri_index_c_0.5_citeseer_highorder_1_x_abs.pkl
File saved at data/adsf/ri_all_c_0.5_citeseer_highorder_1_x_abs.pkl
File saved at data/adsf/ri_index_c_0.5_citeseer_highorder_1_x_abs.pkl
File saved at data/adsf/ri_all_c_0.5_citeseer_highorder_1_x_abs.pkl
File saved at data/adsf/ri_index_c_0.5_citeseer_highorder_1_x_abs.pkl
File saved at data/adsf/ri_all_c_0.5_citeseer_highorder_1_x_abs.pkl
File saved at data/adsf/ri_index_c_0.5_citeseer_highorder_1_x_abs.pkl
File saved at data/adsf/ri_all_c_0.5_citeseer_highorder_1_x_abs.pkl
File saved at data/adsf/ri_index_c_0.5_citeseer_highorder_1_x_abs.pkl
File saved at data/adsf/ri_all_c_0.5_citeseer_highorder_1_x_abs.pkl
File saved at data/adsf/ri_index_c

tensor([[-1.7918, -1.7918, -1.7918, -1.7918, -1.7918, -1.7918],
        [-1.7893, -1.7830, -1.8176, -1.7866, -1.8022, -1.7726],
        [-1.7175, -1.8000, -1.7138, -1.8425, -1.8654, -1.8217],
        ...,
        [-1.8257, -1.7964, -1.7888, -1.7673, -1.7792, -1.7941],
        [-1.7742, -1.7578, -1.8101, -1.8148, -1.7922, -1.8028],
        [-1.7711, -1.8459, -1.7861, -1.7656, -1.7884, -1.7954]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward0>)

In [5]:
def preprocess_features(features):
    """Row-normalize feature matrix and convert to tuple representation"""
    rowsum = np.array(features.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)
    return features.todense(), sparse_to_tuple(features)

def sparse_to_tuple(sparse_mx):
    """Convert sparse matrix to tuple representation."""
    def to_tuple(mx):
        if not sp.isspmatrix_coo(mx):
            mx = mx.tocoo()
        coords = np.vstack((mx.row, mx.col)).transpose()
        values = mx.data
        shape = mx.shape
        return coords, values, shape
features, spars = preprocess_features(features)
